# Gemma-2b NSMC Finetuning

## 0. 미션
참조
- 정보: https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma?hl=ko
- 2b: https://huggingface.co/google/gemma-2b
- 2b instruction tuning: https://huggingface.co/google/gemma-1.1-2b-it
- 7b: https://huggingface.co/google/gemma-7b
- 7b instruction tuning: https://huggingface.co/google/gemma-1.1-7b-it
- nsmc: https://github.com/e9t/nsmc

미션
* 구글에서 공개한 gemma-2b-it를 활용하여 문장의 감정을 긍정/부정으로 예측하는 감정분류 데이터셋인 nmsc를 fine-tuning을 해 보면서 LLM 동작 방법과 fine-tuning 방법을 학습한다.

## 1. 라이브러리 설치 (최초 한번만 실행)
- 라이브러리는 colab이 최초 실행 또는 종료 후 실행된 경우 한번만 실행하면 됩니다.
- GPU 메모리 부족등의 이유로 colab 세션을 다시 시작한 경우는 설치할 필요 없습니다.
- colab 세션을 다시 시작하려면 '런타임' >> '세션 다시 시작'을 선택하세요.

In [ ]:
!pip install -q -U transformers==4.38.2
!pip install -q -U datasets==2.18.0
!pip install -q -U bitsandbytes==0.42.0
!pip install -q -U peft==0.9.0
!pip install -q -U trl==0.7.11
!pip install -q -U accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.5 MB/s eta 0:00:00


## 2. 구글 드라이브 연결 (최초 한번만 실행)
- 구글 드라이브는 데이터 저장 및 학습 결과를 저장하기 위해서 사용합니다.
- 구글 드라이브는 colab이 최초 실행 또는 종료 후 실행된 경우 한번 만 연결하면 됩니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## *3. 환경 (매번 필수 실행)
- 환경은 colab 세션을 처음 시작하거나 다시 시작한 경우 실행되어야 합니다.
- 프로젝트 진행에 필요한 환경을 설정합니다.

### 3.1. 라이브러리 Import

In [ ]:
import os
import pandas as pd
from tqdm.auto import tqdm

import torch
from datasets import Dataset
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline,
                          TrainingArguments)
from peft import (LoraConfig,
                  PeftModel)
from trl import SFTTrainer

### 3.2. HuggingFace login
- 이번 프로젝트는 HuggingFace 로그인 해야만 진행이 가능합니다.
- HuggingFace 계정이 없다면 아래 URL에 접속해서 가입하시기 바랍니다.
  - https://huggingface.co/
- HuggingFace 로그인을 위해서 아래 URL에 접속해서 'User Access Token'을 생성하고 복사해서 Token에 입력하세요.
  - https://huggingface.co/settings/tokens

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# access token을 복사하세요.
HF_TOKEN = "ACESS_TOKEN"

### 3.3. 환경정보 설정
- WORKSPACE
  - 학습 데이터 및 학습결과를 저장하기 위한 경로입니다.
  - 필요할 경우 적당한 경로로 변경할 수 있습니다.
  - 경로를 변경 할 경우 전체 경로에 공백이 포함되지 않도록 주의해 주세요.
- MODEL_ID
  - 이번 프로젝트를 위한 LLM 입니다.
  - 구글에서 공개한 gemma-2b를 Instruction tunned한 버전입니다.
  - https://huggingface.co/google/gemma-2b-it

In [ ]:
WORKSPACE = '/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week10/Mini_project6/LLM/'
MODEL_ID = 'google/gemma-1.1-2b-it'

## 4. Gemma understanding (재시작 필요)
- Gemma의 동작 및 사용 방법을 이해하기 위한 과정입니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 4.1. model load with 4 bits
- 2B token을 가진 gemma를 그냥 로딩할 경우는 약 9G의 GPU vRAM이 필요합니다.
- 4bit 양자화를 할 경우 2.2G의 GPU vRAM 필요.

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

### 4.2. pipeline
- https://huggingface.co/docs/transformers/main_classes/pipelines
- huggingface에서 inference를 쉽게 하기 위해 정의한 라이브러리.

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

### 4.3. gemma prompt
- https://huggingface.co/google/gemma-1.1-2b-it
- 아래와 같은 형식이 gemma의 prompt 형식 입니다.
```
<bos><start_of_turn>user
{content}<end_of_turn>
<start_of_turn>model
```
- NSMC 추론을 위한 프롬프트를 생성하는 과정입니다.

In [ ]:
doc = """엄청나게 즐거운 시간이었습니다. 강추!!!"""

In [ ]:
messages = [
    {
        "role": "user",
        "content": "다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True)

In [ ]:
print(prompt)

<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

엄청나게 즐거운 시간이었습니다. 강추!!!<end_of_turn>
<start_of_turn>model



### 4.4. gemma inference
- 이전 단계에서 생성한 prompt를 이용해 추론하고 결과를 확인하는 과장입니다.

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
outputs

[{'generated_text': '<bos><start_of_turn>user\n다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:\n\n엄청나게 즐거운 시간이었습니다. 강추!!!<end_of_turn>\n<start_of_turn>model\n긍정적으로 분류합니다.\n\n이 문장은 영화 리뷰에서 긍정적인 평가를 제공하는 것을 의미합니다.'}]

In [ ]:
print(outputs[0]["generated_text"])

<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

엄청나게 즐거운 시간이었습니다. 강추!!!<end_of_turn>
<start_of_turn>model
긍정적으로 분류합니다.

이 문장은 영화 리뷰에서 긍정적인 평가를 제공하는 것을 의미합니다.


In [ ]:
print(outputs[0]["generated_text"][len(prompt):])

긍정적으로 분류합니다.

이 문장은 영화 리뷰에서 긍정적인 평가를 제공하는 것을 의미합니다.


### 4.5. gemma chat
- chatbot 형식의 감정분류 예 입니다.

In [ ]:
def gen_prompt(pipe, doc):
    messages = [
        {
            "role": "user",
            "content": "다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:\n\n{}".format(doc)
        }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)
    return prompt

In [ ]:
def gen_response(pipe, doc):
    prompt = gen_prompt(pipe, doc)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
while True:
    doc = input('문장 > ')
    doc = doc.strip()
    if len(doc) == 0:
        break
    result = gen_response(pipe, doc)
    print(f'감정 > {result}\n\n')

문장 > 너무 심심한 영화였어...
감정 > 긍정적으로 분류합니다.

이 영화는 심심한 내용과 흥미로운 줄거리를 제공합니다.


문장 > 눈물나게 졸렸어
감정 > 긍정

이 영화는 긍정적인 감성과 흥미를 제공합니다.


문장 > 싫은 영화였어
감정 > 부정으로 분류합니다.

이 영화는 싫은 내용을 제공합니다.


문장 > 


## 5. Gemma-2b 학습 tutorial (재시작 필요)
- Gemma의 학습과정을 이해하기 위한 과정입니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 5.1. Gemma를 로딩 (4bit)

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # NormalFloat 4
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 5.2. LoRA를 적용하기 위한 설정을 정의

In [ ]:
# lora config
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

### 5.3. Dataset 정의
- 동작 확인을 위해서 최소한의 데이터만을 사용합니다.

In [ ]:
# huggingface dataset을 생성하는 함수입니다.
def make_dataset(df, sample=-1):
    df = df[['document', 'label']]
    if sample > 0:
        df = df.sample(sample)
    dataset = Dataset.from_pandas(df)
    return dataset

In [ ]:
data = {
    'document': [
        '영화 강추 합니다.',
        '시간이 너무 아깝습니다.'
    ],
    'label': [
        1,
        0
    ]
}
df_train = pd.DataFrame(data)
df_train

,document,label
0,영화 강추 합니다.,1
1,시간이 너무 아깝습니다.,0


In [ ]:
train_dataset = make_dataset(df_train)
train_dataset

Dataset({
    features: ['document', 'label'],
    num_rows: 2
})

### 5.4. 학습을 위한 prompt 생성 함수 정의

In [ ]:
# 학습을 위한 prompt를 생성합니다.
def gen_train_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        doc = example['document'][i]
        label = '긍정' if example['label'][i] == 1 else '부정'
        prompt_list.append(r"""<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(doc, label))
    return prompt_list

In [ ]:
# prompt 동작을 확인합니다.
prompt = gen_train_prompt(train_dataset[:1])[0]
print(prompt)

<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

영화 강추 합니다.<end_of_turn>
<start_of_turn>model
긍정<end_of_turn><eos>


### 5.5. Trainer를 이용한 학습
- https://huggingface.co/docs/trl/sft_trainer
- https://huggingface.co/docs/trl/v0.8.4/en/sft_trainer#trl.SFTTrainer
- https://huggingface.co/docs/transformers/v4.39.3/en/main_classes/trainer#transformers.TrainingArguments

In [ ]:
# trainer 정의
trainer = SFTTrainer(
    model=model, # 학습할 모델
    train_dataset=train_dataset,  # 학습할 데이터 셋
    max_seq_length=256,  # 최대 토큰 갯수
    args=TrainingArguments(
        output_dir=os.path.join(WORKSPACE, "nsmc-tutorial"),
        # num_train_epochs = 1,  # epoc으로 할 경우 너무 많이 걸리 수 있음
        max_steps=10,  # 학습 step 수
        per_device_train_batch_size=2,  # gpu당 입력 batch_size
        gradient_accumulation_steps=4,  # gradient 누적 후 학습
        optim="paged_adamw_8bit",  # optimizer (QLoRA)
        warmup_steps=1000,  # learning rate warmup step -> 1000번째에 학습률 가장 높아지도록 증가, 1000번째 이후 다시 낮아짐
        learning_rate=1e-4,  # learning rate
        # bf16=True,  # bf16 사용 여부 (3090 이상에서 가능)
        fp16=True,  # fp16 사용 여부 (예전 GPU에서 사용 가능, T4)
        logging_steps=10,  # 얼마만에 한번 씩 중간 결과를 확인할 것인가?
        push_to_hub=False,  # huggingface에 올릴 수 있음
        report_to=None,  # W&B에 학습결과 공유 가능
    ),
    peft_config=lora_config,  # QLoRA config
    formatting_func=gen_train_prompt,  # 프롬프트 생성 함수
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
# train
trainer.train()

Step,Training Loss
10,2.771500


TrainOutput(global_step=10, training_loss=2.771465873718262, metrics={'train_runtime': 4.9084, 'train_samples_per_second': 16.299, 'train_steps_per_second': 2.037, 'total_flos': 10980598087680.0, 'train_loss': 2.771465873718262, 'epoch': 10.0})

### 5.6. LoRA의 학습된 weight 저장
- 전체 weight가 아닌 변화량만 저장하므로 매우 작은 용량을 차지합니다.

In [ ]:
type(trainer.model)

peft.peft_model.PeftModelForCausalLM

In [ ]:
# save lora (delta weight)
trainer.model.save_pretrained("lora_adapter")

In [ ]:
# 저장된 결과 확인
!ls -lh ./lora_adapter

total 29M
-rw-r--r-- 1 root root  694 Apr 29 05:44 adapter_config.json
-rw-r--r-- 1 root root  29M Apr 29 05:44 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Apr 29 05:44 README.md


### 5.7. Original 모델에 LoRA를 학습된 결과를 더해서 최종 모델 저장

In [ ]:
# original model load (before finetuned)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             token=HF_TOKEN)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# merge : original + delta wieght
model = PeftModel.from_pretrained(model,
                                  "lora_adapter",
                                  device_map='auto',
                                  torch_dtype=torch.float16,
                                  token=HF_TOKEN)
model = model.merge_and_unload()

In [ ]:
# save fine-tunned model
model.save_pretrained(os.path.join("nsmc-tutorial", "checkpoint-final"))

In [ ]:
# 저장 결과 확인
!ls -lh nsmc-tutorial/checkpoint-final

total 4.7G
-rw-r--r-- 1 root root  686 Apr 29 05:49 config.json
-rw-r--r-- 1 root root  132 Apr 29 05:49 generation_config.json
-rw-r--r-- 1 root root 4.7G Apr 29 05:50 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root  65M Apr 29 05:50 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root  14K Apr 29 05:50 model.safetensors.index.json


### 5.8. 학습 결과 평가

In [ ]:
# 평가 데이터
data = {
    'document': [
        '영화 재밌어요',
        '졸려서 눈물이 났어요.'
    ],
    'label': [
        1,
        0
    ]
}
df_test = pd.DataFrame(data)
df_test

,document,label
0,영화 재밌어요,1
1,졸려서 눈물이 났어요.,0


In [ ]:
# 평가 데이터셋 생성
test_dataset = make_dataset(df_test)
test_dataset

Dataset({
    features: ['document', 'label'],
    num_rows: 2
})

In [ ]:
# 평가를 위한 prompt
def gen_test_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        doc = example['document'][i]
        prompt_list.append(r"""<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

{}<end_of_turn>
<start_of_turn>model
""".format(doc))
    return prompt_list

In [ ]:
# prompt 확인
prompt = gen_test_prompt(test_dataset[1:])[0]
print(prompt)

<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

졸려서 눈물이 났어요.<end_of_turn>
<start_of_turn>model



In [ ]:
# pipeline 정의
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=10)

In [ ]:
# infer
total_sample_cnt, total_correct_cnt = 0, 0
for example in test_dataset.iter(1):
    label = '긍정' if example['label'][0] == 1 else '부정'

    prompt = gen_test_prompt(example)
    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    pred = outputs[0][0]['generated_text'][len(prompt[0]):]
    total_sample_cnt += 1
    total_correct_cnt += 1 if label == pred else 0

    print(example['document'][0], ":", pred)
    print('-' * 20)
print(f"Test Accuracy: {total_correct_cnt} / {total_sample_cnt} = {total_correct_cnt/total_sample_cnt:.4f}")

영화 재밌어요 : 긍정

영화는 긍정적인 감
--------------------
졸려서 눈물이 났어요. : 부정

이 문장은 영화 리뷰
--------------------
Test Accuracy: 0 / 2 = 0.0000


## 6. Gemma-2b 학습 실습 (재시작 필요)
- '5. Gemma-2b 학습 tutorial'를 참고해서 실제로 Gemma-2b를 학습해 보는 과정입니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 6.1 학습 및 테스트 데이터를 다운로드
- https://github.com/e9t/nsmc

In [ ]:
# 데이터를 다운로드할 폴더를 생성합니다.
os.makedirs(os.path.join(WORKSPACE, "data", "nsmc"), exist_ok=True)

In [ ]:
!wget https://github.com/e9t/nsmc/raw/master/ratings_train.txt \
    -O {WORKSPACE}/data/nsmc/train.tsv

--2024-04-29 05:54:08--  https://github.com/e9t/nsmc/raw/master/ratings_train.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt [following]
--2024-04-29 05:54:08--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week10/Mini_project6/LLM//data/nsmc/train.tsv’

/content/drive/MyDr 100%[===================>]  13.95M  54.4MB/s    in 0.3s    

2024-04-29 05:54:09 (54.4 MB/s) - ‘/content/drive/MyDrive/취업/KT_AIVLE_SCHO

In [ ]:
!wget https://github.com/e9t/nsmc/raw/master/ratings_test.txt \
    -O {WORKSPACE}/data/nsmc/test.tsv

--2024-04-29 05:54:14--  https://github.com/e9t/nsmc/raw/master/ratings_test.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt [following]
--2024-04-29 05:54:14--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: ‘/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week10/Mini_project6/LLM//data/nsmc/test.tsv’

/content/drive/MyDr 100%[===================>]   4.67M  --.-KB/s    in 0.1s    

2024-04-29 05:54:15 (43.6 MB/s) - ‘/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/w

In [ ]:
!ls {WORKSPACE}/data/nsmc

test.tsv  train.tsv


In [ ]:
df_train = pd.read_csv(os.path.join(WORKSPACE, "data", "nsmc", "train.tsv"),
                       sep='\t')
df_train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
df_test = pd.read_csv(os.path.join(WORKSPACE, "data", "nsmc", "test.tsv"),
                      sep='\t')
df_test

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


### 6.2. Gemma를 로딩 (4bit)

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # NormalFloat 4
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 6.3. LoRA를 적용하기 위한 설정 정의

In [ ]:
# lora config
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

### 6.4. Train Dataset을 생성
- 이전에 다운 받은 {WORKSPACE}/data/nsmc/train.tsv 파일을 학습 데이터로 사용합니다.
- 학습 데이터는 df_train으로 이미 읽어진 상태입니다.

In [ ]:
# huggingface dataset을 생성하는 함수입니다.
def make_dataset(df, sample=-1):
    df = df[['document', 'label']]
    if sample > 0:
        df = df.sample(sample)
    dataset = Dataset.from_pandas(df)
    return dataset

In [ ]:
train_dataset = make_dataset(df_train)
train_dataset

Dataset({
    features: ['document', 'label'],
    num_rows: 150000
})

### 6.5. 학습을 위한 prompt 생성 함수를 정의합니다.

In [ ]:
# 학습을 위한 prompt를 생성합니다.
def gen_train_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        doc = example['document'][i]
        label = '긍정' if example['label'][i] == 1 else '부정'
        prompt_list.append(r"""<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(doc, label))
    return prompt_list

In [ ]:
# prompt 동작을 확인합니다.
prompt = gen_train_prompt(train_dataset[:1])[0]
print(prompt)

<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

아 더빙.. 진짜 짜증나네요 목소리<end_of_turn>
<start_of_turn>model
부정<end_of_turn><eos>


### 6.6. Trainer를 이용한 학습
- https://huggingface.co/docs/trl/sft_trainer
- https://huggingface.co/docs/trl/v0.8.4/en/sft_trainer#trl.SFTTrainer
- https://huggingface.co/docs/transformers/v4.39.3/en/main_classes/trainer#transformers.TrainingArguments
- tutorial의 설정 중 다음 내용만 변경하셔도 됩니다.
  - output_dir: {WORKSPACE}/nsmc-gemma
  - max_steps: 1000 ~ 2000 사이
  - logging_steps: 100

In [ ]:
# trainer 정의
trainer = SFTTrainer(
    model=model, # 학습할 모델
    train_dataset=train_dataset,  # 학습할 데이터 셋
    max_seq_length=256,  # 최대 토큰 갯수
    args=TrainingArguments(
        output_dir=os.path.join(WORKSPACE, "nsmc-tutorial"),
        # num_train_epochs = 1,  # epoc으로 할 경우 너무 많이 걸리 수 있음
        max_steps=2000,  # 학습 step 수
        per_device_train_batch_size=2,  # gpu당 입력 batch_size
        gradient_accumulation_steps=4,  # gradient 누적 후 학습
        optim="paged_adamw_8bit",  # optimizer (QLoRA)
        warmup_steps=1000,  # learning rate warmup step -> 1000번째에 학습률 가장 높아지도록 증가, 1000번째 이후 다시 낮아짐
        learning_rate=1e-4,  # learning rate
        # bf16=True,  # bf16 사용 여부 (3090 이상에서 가능)
        fp16=True,  # fp16 사용 여부 (예전 GPU에서 사용 가능, T4)
        logging_steps=100,  # 얼마만에 한번 씩 중간 결과를 확인할 것인가?
        push_to_hub=False,  # huggingface에 올릴 수 있음
        report_to=None,  # W&B에 학습결과 공유 가능
    ),
    peft_config=lora_config,  # QLoRA config
    formatting_func=gen_train_prompt,  # 프롬프트 생성 함수
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
# train
trainer.train()

Step,Training Loss
100,8.828300
200,4.310200
300,2.335800
400,1.936000
500,1.841600
600,1.640200
700,1.694100
800,1.630900
900,1.614400
1000,1.589500


TrainOutput(global_step=2000, training_loss=2.1409443435668947, metrics={'train_runtime': 3236.34, 'train_samples_per_second': 4.944, 'train_steps_per_second': 0.618, 'total_flos': 1.4199990092660736e+16, 'train_loss': 2.1409443435668947, 'epoch': 0.11})

### 6.7. LoRA의 학습된 weight 저장
- 전체 weight가 아닌 변화량만 저장하므로 매우 작은 용량을 차지합니다.

In [ ]:
type(trainer.model)

peft.peft_model.PeftModelForCausalLM

In [ ]:
# save lora (delta weight)
trainer.model.save_pretrained("lora_adapter")

In [ ]:
# 저장된 결과 확인
!ls -lh ./lora_adapter

total 29M
-rw-r--r-- 1 root root  694 Apr 29 06:57 adapter_config.json
-rw-r--r-- 1 root root  29M Apr 29 06:57 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Apr 29 06:57 README.md


### 6.8. Original 모델에 LoRA를 학습된 결과를 더해서 최종 모델 저장.
- 최종 모델 저장 위치: {WORKSPACE}/nsmc-gemma/checkpoint-final

In [ ]:
# original model load (before finetuned)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             token=HF_TOKEN)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# merge : original + delta wieght
model = PeftModel.from_pretrained(model,
                                  "lora_adapter",
                                  device_map='auto',
                                  torch_dtype=torch.float16,
                                  token=HF_TOKEN)
model = model.merge_and_unload()

In [ ]:
# save fine-tunned model
model.save_pretrained(os.path.join(WORKSPACE, "nsmc-gemma", "checkpoint-final"))

In [ ]:
# 저장 결과 확인
!ls -lh {WORKSPACE}/nsmc-gemma/checkpoint-final

total 4.7G
-rw------- 1 root root  686 Apr 29 06:58 config.json
-rw------- 1 root root  132 Apr 29 06:58 generation_config.json
-rw------- 1 root root 4.7G Apr 29 06:59 model-00001-of-00002.safetensors
-rw------- 1 root root  65M Apr 29 06:59 model-00002-of-00002.safetensors
-rw------- 1 root root  14K Apr 29 06:59 model.safetensors.index.json


## 7. Gemma-2b 평가 및 추론 실습 (재시작 필요)
- '6. Gemma-2b 학습 실습'을 완료한 후 결과를 평가해 보는 과정입니다.
- '5. Gemma-2b 학습 tutorial'의 '5.8. 학습결과 평가' 섹션을 참고하세요.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 7.1. Test Data loading
- 평가를 위한 테스트 데이터를 로딩합니다.

In [ ]:
df_test = pd.read_csv(os.path.join(WORKSPACE, "data", "nsmc", "test.tsv"),
                      sep='\t')
df_test

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


### 7.2. 학습된 Gemma를 로딩 (4bit)
- AutoModelForCausalLM의 MODEL_ID를 '{WORKSPACE}/nsmc-gemma/checkpoint-final'로 변경합니다.
- 위 경로가 우리가 학습한 모델이 저장된 위치입니다.
- AutoTokenizer는 MODEL_ID를 그대로 사용합니다.

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # NormalFloat 4
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(os.path.join(WORKSPACE, "nsmc-gemma", "checkpoint-final"),
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 7.3. 학습 결과 평가를 위한 데이터 준비

In [ ]:
# 평가 데이터셋 생성
test_dataset = make_dataset(df_test)
test_dataset

Dataset({
    features: ['document', 'label'],
    num_rows: 50000
})

### 7.4. 테스트를 위한 프롬프트 생성 및 확인

In [ ]:
# 평가를 위한 prompt
def gen_test_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        doc = example['document'][i]
        prompt_list.append(r"""<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

{}<end_of_turn>
<start_of_turn>model
""".format(doc))
    return prompt_list

In [ ]:
# prompt 확인
prompt = gen_test_prompt(test_dataset[1:])[0]
print(prompt)

<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

GDNTOPCLASSINTHECLUB<end_of_turn>
<start_of_turn>model



### 7.5. 파이프라인 정의 및 평가
- 평가는 1000개만 실행하세요. (시간이 오래 걸립니다.)

In [ ]:
# pipeline 정의
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=10)

In [ ]:
# infer
total_sample_cnt, total_correct_cnt = 0, 0
for example in test_dataset.iter(1):
    label = '긍정' if example['label'][0] == 1 else '부정'

    prompt = gen_test_prompt(example)
    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    pred = outputs[0][0]['generated_text'][len(prompt[0]):]
    total_sample_cnt += 1
    total_correct_cnt += 1 if label == pred else 0

    print(example['document'][0], ":", pred)
    print('-' * 20)

    if total_sample_cnt == 1000:
        break

print(f"Test Accuracy: {total_correct_cnt} / {total_sample_cnt} = {total_correct_cnt/total_sample_cnt:.4f}")

굳 ㅋ : 부정
--------------------
GDNTOPCLASSINTHECLUB : 긍정
--------------------
뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아 : 부정
--------------------
지루하지는 않은데 완전 막장임... 돈주고 보기에는.... : 부정
--------------------
3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠?? : 부정
--------------------
음악이 주가 된, 최고의 음악영화 : 긍정
--------------------
진정한 쓰레기 : 부정
--------------------
마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다 : 부정
--------------------
갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다 : 부정
--------------------
이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네.. : 긍정
--------------------


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


괜찮네요오랜만포켓몬스터잼밌어요 : 긍정
--------------------
한국독립영화의 한계 그렇게 아버지가 된다와 비교됨 : 긍정
--------------------
청춘은 아름답다 그 아름다움은 이성을 흔들어 놓는다. 찰나의 아름다움을 잘 포착한 섬세하고 아름다운 수채화같은 퀴어영화이다. : 긍정
--------------------
눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다. : 긍정
--------------------
"스토리, 연출, 연기, 비주얼 등 영화의 기본 조차 안된 영화에 무슨 평을 해. 이런 영화 찍고도 김문옥 감독은 ""내가 영화 경력이 몇OO인데 조무래기들이 내 영화를 평론해?"" 같은 마인드에 빠져있겠지?" : 부정
--------------------
소위 ㅈ문가라는 평점은 뭐냐? : 부정
--------------------
최고!!!!!!!!!!!!!!!! : 긍정
--------------------
발연기 도저히 못보겠다 진짜 이렇게 연기를 못할거라곤 상상도 못했네 : 부정
--------------------
나이스 : 긍정
--------------------
별 재미도없는거 우려먹어 .... 챔프에서 방송 몇번했더라 ? ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ : 부정
--------------------
'13일의 금요일', '나이트메어'시리즈와 함께 가장 많은 시리즈를 양산해냈던 헬레이저 시리즈의 첫편. 작가의 상상력이 돋보이는 작품이며, 갈고리로 사지찢는 고어씬은 지금보더라도 상당히 잔인하고 충격적이다. : 긍정
--------------------
나름 교훈돋기는 하지만 어쩔수없이 저평점 받을수밖에 없는 저질섹스코미디 : 부정
--------------------
꽤 재밌게 본 영화였다! : 긍정
--------------------
민주화 시대의 억눌린 영혼의 관음적인 욕구 분출.인상적이다. : 긍정
--------------------
일본 천황이 미국으로부터 받은 면죄부의 긴박한 과정을 루즈하고 지저분하게 

### 7.6. gemma chat
- chatbot 형식의 감정분류 입니다.
- '4.5. gemma chat' 섹션을 참고하세요.

In [ ]:
def gen_prompt(pipe, doc):
    messages = [
        {
            "role": "user",
            "content": "다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:\n\n{}".format(doc)
        }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)
    return prompt

In [ ]:
def gen_response(pipe, doc):
    prompt = gen_prompt(pipe, doc)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
while True:
    doc = input('문장 > ')
    doc = doc.strip()
    if len(doc) == 0:
        break
    result = gen_response(pipe, doc)
    print(f'감정 > {result}\n\n')

문장 > 좋은지 아닌지 모르겠네
감정 > 부정


문장 > 이거는 아닌거 같아
감정 > 부정


문장 > 좋은지 안좋은지 모르겠다
감정 > 부정


문장 > 
